In [74]:
# !pip install --upgrade tfds-nightly

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os

In [76]:
labels_list = [
    'одуванчик',
    'маргаритка'
    'тюльпаны',
    'подсолнухи',
    'розы',
    ]

In [55]:
test_set, train_set = tfds.load(
                                'tf_flowers',
                                split = ["train[0%:20%]", "train[20%:]"], 
                                as_supervised = True,
                                batch_size = 32,
                                # with_info = True
                                )
classes_number = 5

In [58]:
from tensorflow.keras.layers import RandomRotation, RandomFlip,\
Resizing, Rescaling, RandomContrast, RandomZoom, RandomCrop

IMG_SIZE = 224

resize_and_rescale = tf.keras.Sequential([
  Resizing(IMG_SIZE, IMG_SIZE),
  Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([                   
  RandomFlip("horizontal_and_vertical"),
  # RandomCrop(IMG_SIZE//2, IMG_SIZE//2),
  RandomRotation(0.2),
  RandomContrast(0.2),
  # RandomZoom(0.5)
])

augmentation=tf.keras.Sequential([
  resize_and_rescale,
  data_augmentation
])



In [60]:
train_set = train_set.map(lambda x, y: (augmentation(x, training=True), 
                                        tf.one_hot(y, classes_number)))
valid_set = valid_set.map(lambda x, y: (resize_and_rescale(x), tf.one_hot(y, classes_number)))

In [64]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, MaxPooling2D


inputs = Input((IMG_SIZE, IMG_SIZE,3))

model = MobileNet(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top = False,
                  weights = 'imagenet')


model.trainable = False
x=model(inputs)
x = MaxPooling2D()(x)
global_average_layer = GlobalAveragePooling2D()
x = global_average_layer(x)
outputs = Dense(classes_number, activation = 'softmax')(x)

model = tf.keras.models.Model(inputs, outputs)

model.summary()

17235968/17225924 [==============================] - 6s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 3, 1024)       0         
 )                                                               
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 5)                 5125   

In [65]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [66]:
modelname = 'model'
models_dir = 'models'
os.makedirs(models_dir, exist_ok = True)
best_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = os.path.join('models', f'{modelname}_best.hdf5'),
                                   monitor = 'val_loss',
                                   save_best_only = True,
                                   mode = 'min')
last_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = os.path.join('models', f'{modelname}_last.hdf5'),
                                   monitor = 'val_loss',
                                   save_best_only = False,
                                   mode = 'auto')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
callbacks = [best_checkpoint, last_checkpoint, early_stopping]

In [67]:
epochs=30

history = model.fit(train_set,
                    epochs=epochs,
                    validation_data=valid_set,
                    callbacks = callbacks,
                    )

Epoch 1/30
92/92 [==============================] - 56s 518ms/step - loss: 1.2901 - acc: 0.5501 - val_loss: 0.7567 - val_acc: 0.7425
Epoch 2/30
92/92 [==============================] - 45s 487ms/step - loss: 0.7166 - acc: 0.7316 - val_loss: 0.6547 - val_acc: 0.7820
Epoch 3/30
92/92 [==============================] - 44s 480ms/step - loss: 0.6013 - acc: 0.7888 - val_loss: 0.6519 - val_acc: 0.7888
Epoch 4/30
92/92 [==============================] - 44s 480ms/step - loss: 0.5471 - acc: 0.8035 - val_loss: 0.5758 - val_acc: 0.8215
Epoch 5/30
92/92 [==============================] - 46s 506ms/step - loss: 0.5126 - acc: 0.8161 - val_loss: 0.5578 - val_acc: 0.8243
Epoch 6/30
92/92 [==============================] - 45s 489ms/step - loss: 0.4747 - acc: 0.8297 - val_loss: 0.5619 - val_acc: 0.8324
Epoch 7/30
92/92 [==============================] - 42s 463ms/step - loss: 0.4744 - acc: 0.8277 - val_loss: 0.5624 - val_acc: 0.8338
Epoch 8/30
92/92 [==============================] - 44s 475ms/step - 